In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from dataset.dataset import MSGDataModule
from models.FNO import FNO2d
from models.LightningModule import LitEstimator

In [5]:
ckpt_fn = './SIS_estimation/u2n8v4h9/checkpoints/epoch=19-step=23040.ckpt'

model = LitEstimator.load_from_checkpoint(ckpt_fn)

# disable randomness, dropout, etc...
model.eval()

# predict with the model
y_hat = model(x)


TypeError: __init__() missing 1 required positional argument: 'model'